<a href="https://colab.research.google.com/github/AnzorGozalishvili/NASA_ODSR_DATA/blob/main/NASA_OSDR_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [3]:
import datasets

In [4]:
from datasets import load_dataset

In [9]:
dataset = load_dataset('anz2/NASA_OSDR')

In [17]:
dataset['train'][10]

{'Sample Name': 'GSM2684068',
 'Protocol REF': 'Nucleic Acid Extraction',
 'Parameter Value: DNA Fragmentation': 'sonication',
 'Parameter Value: DNA Fragment Size': '200-300 base pair',
 'Extract Name': 'GSM2684068',
 'Protocol REF.1': 'Library Construction',
 'Parameter Value: Library Strategy': 'BisPCR2',
 'Parameter Value: Library Selection': 'other',
 'Parameter Value: Library Layout': 'PAIRED',
 'Protocol REF.2': 'Nucleic Acid Sequencing',
 'Parameter Value: Sequencing Instrument': 'Illumina MiSeq',
 'Assay Name': 'BisPCR2',
 'Parameter Value: Read Length': '150 base pair',
 'Raw Data File': 'GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz, GLDS-524_wgbs_GSM2684068_R2_raw.fastq.gz',
 'Protocol REF.3': 'GeneLab raw data processing protocol',
 'Parameter Value: Read Depth': '199027 read',
 'Parameter Value: MultiQC File Names': 'GLDS-524_Gwgbs_raw_multiqc_report.zip'}

In [18]:
fasta_file_names = dataset['train'][10]['Raw Data File']

In [19]:
fasta_file_names

'GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz, GLDS-524_wgbs_GSM2684068_R2_raw.fastq.gz'

In [ ]:
!pip install boto3

In [38]:
import boto3
import os
from botocore import UNSIGNED
from botocore.client import Config

In [39]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [43]:
NASA_OSDR_BUCKET = 'nasa-osdr'
DOWNLOAD_DIR = 'nasa_osdr_fasta_files'
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

In [53]:
def download_fasta_file_from_s3(fasta_filename):
    response = s3.list_objects_v2(Bucket=NASA_OSDR_BUCKET, Prefix='OSD-524')

    if 'Contents' in response:
        for obj in response['Contents']:
            key = obj['Key']
            if key.endswith(fasta_filename):
                file_path = os.path.join(DOWNLOAD_DIR, fasta_filename)
                s3.download_file(Bucket=NASA_OSDR_BUCKET, Key=key, Filename=file_path)
                print('file downloaded to path:', file_path)

In [45]:
fasta_file_names.split(',')[0]

'GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz'

In [54]:
download_fasta_file_from_s3(fasta_file_names.split(',')[0])

file downloaded to path: nasa_osdr_fasta_files/GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz
file downloaded to path: nasa_osdr_fasta_files/GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz


In [55]:
!ls nasa_osdr_fasta_files

GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz


In [58]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.7 MB/s eta 0:00:00


In [66]:
from Bio import SeqIO
import gzip

In [79]:
def load_rna_sequences_from_fasta(fasta_file):
    with gzip.open(fasta_file, "rt") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            yield record

In [80]:
fasta_loader = load_rna_sequences_from_fasta('nasa_osdr_fasta_files/GLDS-524_wgbs_GSM2684068_R1_raw.fastq.gz')
sample = next(iter(fasta_loader))

In [82]:
sample.__dict__

{'_seq': Seq('GTTTTTTTTAAAGTGTTAGATAGTAGTTAAAATAAGAGATATGTTTTTTATTTA...AGG'),
 'id': 'SRR5756273.1',
 'name': 'SRR5756273.1',
 'description': 'SRR5756273.1 1 length=151',
 'dbxrefs': [],
 'annotations': {},
 '_per_letter_annotations': {'phred_quality': [32,
   32,
   32,
   32,
   32,
   32,
   35,
   35,
   35,
   31,
   16,
   16,
   35,
   38,
   35,
   38,
   38,
   37,
   38,
   37,
   38,
   34,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   38,
   37,
   38,
   37,
   37,
   37,
   33,
   38,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   38,
   38,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   38,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   36,
   35,
   37,
   38,
   38,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   38,
   38,
   38,
   38,
   38,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   37,
   32,
   38,
   34,
   30,
   36,
   39,


In [83]:
!git clone https://github.com/mana438/RNABERT.git

Cloning into 'RNABERT'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 80 (delta 31), reused 66 (delta 17), pack-reused 0
Receiving objects: 100% (80/80), 347.43 KiB | 3.66 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [89]:
# !apt-get install -y g++
# !apt-get install build-essential
# !cd RNABERT && python setup.py install

In [90]:
!ls

nasa_osdr_fasta_files  RNABERT	sample_data


In [91]:
%%writefile RNABERT/module.py
import torch
# import alignment_C as Aln_C
from torch import nn
import torch.nn.functional as F
import torch.multiprocessing as mp
from dataload import  num_to_base
import time
import numpy as np
class Train_Module:
	def __init__(self, config):
		self.device = "cuda" if torch.cuda.is_available() else "cpu"
		self.config = config

	def train_MLM(self, low_seq_0, masked_seq_0, prediction_scores):
		criterion = nn.CrossEntropyLoss()
		mask = masked_seq_0 - low_seq_0 != 0
		length_mask = masked_seq_0 != 0
		same_base_mask = torch.bernoulli(torch.ones(mask.shape)*0.05).byte().to(self.device)
		mask = mask + same_base_mask
		index = torch.nonzero(mask).split(1, dim=1)
		prediction_scores = torch.squeeze(prediction_scores[index])
		new_low_seq_0 = torch.squeeze(low_seq_0[index])
		loss = criterion(prediction_scores, new_low_seq_0)
		_, preds = torch.max(prediction_scores, 1)

		correct = torch.sum(preds == new_low_seq_0.data).double()/len(new_low_seq_0)
		return loss, correct

	def train_SSL(self, SS, prediction_scores):
		criterion = nn.CrossEntropyLoss()
		index = torch.nonzero(SS).split(1, dim=1)
		prediction_scores = torch.squeeze(prediction_scores[index])
		SS_answer = torch.squeeze(SS[index])

		loss = criterion(prediction_scores, SS_answer)
		_, preds = torch.max(prediction_scores, 1)
		correct = torch.sum(preds == SS_answer.data).double()/len(SS_answer)
		return loss, correct

	def train_SFP(self, low_seq_0, seq_len_0, low_seq_1, seq_len_1, family_0, family_1, z0_list, z1_list):
		distance_label = torch.where((family_0 - family_1) == 0,  torch.ones(family_0.size()[0]), torch.full((family_0.size()[0],), -1) ).to("cuda")
		distance_label_same = torch.where((family_0 - family_1) == 0,  torch.ones(family_0.size()[0]), torch.zeros(family_0.size()[0]) ).to("cuda")
		distance_label_diff = torch.where((family_0 - family_1) != 0,  torch.ones(family_0.size()[0]), torch.zeros(family_0.size()[0]) ).to("cuda")
		_, logits = self.match(z0_list, z1_list)
		distance = -logits
		distance_same = torch.dot(distance_label_same , distance.view(-1)) / torch.sum(distance_label_same)
		distance_diff = torch.dot(distance_label_diff , distance.view(-1)) / torch.sum(distance_label_diff)
		margin = 600
		loss = F.relu( distance_same - distance_diff + margin )

		# accuracy
		predicted_label = torch.where(distance.view(-1) < 660,  torch.ones(family_0.size()[0]).to("cuda"), torch.full((family_0.size()[0],), -1).to("cuda"))
		match = torch.where((distance_label * predicted_label) == 1, torch.ones(family_0.size()[0]).to("cuda"), torch.zeros(family_0.size()[0]).to("cuda"))
		correct = torch.sum(match)
		return loss, correct

	def train_MUL(self, z0_list, z1_list, common_index_0, common_index_1, seq_len_0, seq_len_1):
		bert_scores, _ = self.match(z0_list, z1_list)
		loss = 0.0
		for i, bert_score in enumerate(bert_scores):
			loss += self.structural_learning(bert_score, common_index_0[i], seq_len_0[i], common_index_1[i], seq_len_1[i])
		return loss

	def test_align(self, low_seq_0, low_seq_1, z0_list, z1_list, common_index_0, common_index_1, seq_len_0, seq_len_1, show_aln):
		bert_scores, _ = self.match(z0_list, z1_list)
		sequence_a = num_to_base(low_seq_0)
		sequence_b = num_to_base(low_seq_1)
		len_pred_match = 0
		len_ref_match = 0
		len_TP = 0
		for i, bert_score in enumerate(bert_scores):
			bert_score = torch.flatten(bert_score.T).tolist()
			x = 1 if show_aln == True else 0
			common_index_A_B = Aln_C.global_aln(bert_score, [0] * len(bert_score), [0] * len(bert_score), sequence_a[i], sequence_b[i], seq_len_0[i], seq_len_1[i], self.config.gap_opening, self.config.gap_extension, x, 0)
			common_index_A_B = torch.tensor(common_index_A_B).to(self.device).view(2, -1)
			len_pred_match += int(torch.sum(common_index_A_B[0]))
			len_ref_match += int(torch.sum(common_index_0[i]))
			a = torch.flatten( (common_index_A_B[0] == 1).nonzero()*10000 + (common_index_A_B[1] == 1).nonzero()).tolist()
			b = torch.flatten( (common_index_0[i] == 1).nonzero()*10000 + (common_index_1[i] == 1).nonzero()).tolist()
			len_TP += len(set(a) & set(b))
		return len_TP, len_pred_match, len_ref_match

	def structural_learning(self, bert_score, common_index_0, seq_len_0, common_index_1, seq_len_1):
		reference_alignment_score = self.calc_reference_alignment_score(bert_score, common_index_0, seq_len_0, common_index_1, seq_len_1)
		prediction_alignment_score, _ = self.calc_prediction_alignment_score(bert_score, common_index_0, seq_len_0, common_index_1, seq_len_1)
		return prediction_alignment_score - reference_alignment_score

	def match_bert_score(self, bert_score, common_index_0, seq_len_0, common_index_1, seq_len_1):
		index_0 = (common_index_0 == 1).nonzero()
		index_1 = (common_index_1 == 1).nonzero()
		index = torch.cat([index_0, index_1], axis=1).T
		index = tuple(index)
		omega = bert_score[index]
		return omega

	def margin_matrix(self, bert_score, common_index_0, seq_len_0, common_index_1, seq_len_1):
		index_0 = (common_index_0 == 1).nonzero()
		index_1 = (common_index_1 == 1).nonzero()
		index = torch.cat([index_0, index_1], axis=1).T
		index = tuple(index)
		margin_mat_FP = torch.ones(bert_score.shape).to(self.device) * self.config.margin_FP
		margin_mat_FP[index] = 0.0
		margin_mat_FN = torch.zeros(bert_score.shape).to(self.device)
		margin_mat_FN[index] = self.config.margin_FN
		return margin_mat_FP, margin_mat_FN

	def margin_score(self, common_index_A_B, common_index_0, common_index_1):
		len_pred_match = int(torch.sum(common_index_A_B[0]))
		len_ref_match = int(torch.sum(common_index_0))
		a = torch.flatten( (common_index_A_B[0] == 1).nonzero()*10000 + (common_index_A_B[1] == 1).nonzero()).tolist()
		b = torch.flatten( (common_index_0 == 1).nonzero()*10000 + (common_index_1 == 1).nonzero()).tolist()
		len_TP = len(set(a) & set(b))
		len_FP = len_pred_match - len_TP
		len_FN = len_ref_match - len_TP
		return len_FP * self.config.margin_FP + len_FN * self.config.margin_FN

	def calc_reference_alignment_score(self, bert_score, common_index_0, seq_len_0, common_index_1, seq_len_1):
		reference_alignment_score = 0
		reference_alignment_score += self.gapscore(common_index_0, seq_len_0)
		reference_alignment_score += self.gapscore(common_index_1, seq_len_1)
		omega = self.match_bert_score(bert_score, common_index_0, seq_len_0, common_index_1, seq_len_1)
		reference_alignment_score += torch.sum(omega)
		return reference_alignment_score

	def calc_prediction_alignment_score(self, bert_score, common_index_0, seq_len_0, common_index_1, seq_len_1):
		sequence_a = "N"*seq_len_0
		sequence_b = "N"*seq_len_1
		margin_mat_FP, margin_mat_FN = self.margin_matrix(bert_score, common_index_0, seq_len_0, common_index_1, seq_len_1)
		common_index_A_B = Aln_C.global_aln(torch.flatten(bert_score.T).tolist(), torch.flatten(margin_mat_FP.T).tolist(), torch.flatten(margin_mat_FN.T).tolist(), sequence_a, sequence_b, seq_len_0, seq_len_1, self.config.gap_opening, self.config.gap_extension, 0, 0)
		common_index_A_B = torch.tensor(common_index_A_B).to(self.device).view(2, -1)
		prediction_alignment_score = self.calc_reference_alignment_score(bert_score, common_index_A_B[0], seq_len_0, common_index_A_B[1], seq_len_1) + self.margin_score(common_index_A_B, common_index_0, common_index_1)
		return prediction_alignment_score, common_index_A_B

	def gapscore(self, index, seq_len):
		seq_len = int(seq_len)
		index = index[:seq_len].to('cpu').detach().numpy().copy()
		all_zeros = seq_len - np.count_nonzero(index)
		extend_zeros = seq_len - np.count_nonzero(np.insert(index[:-1], 0, 1) + index)
		open_zeros = all_zeros - extend_zeros
		return (-1*self.config.gap_opening + -1*self.config.gap_extension ) * open_zeros + -1*self.config.gap_extension * extend_zeros

	def match(self,z0_list, z1_list):
		match_scores = []
		logits = []
		for z0, z1 in zip(z0_list, z1_list):
			# cos similarity
			match_score = nn.CosineSimilarity(dim=2, eps=1e-6)(z0.unsqueeze(1).repeat(1, z1.shape[0],1) , z1.unsqueeze(0).repeat(z0.shape[0], 1,1))
			s = 1.3 * match_score
			# L1 distance
			# s = -torch.sum(torch.abs(z0.unsqueeze(1)-z1), -1)
			# match_score = torch.exp(s)

			# soft align
			a, b = F.softmax(s, 1), F.softmax(s, 0)
			c = a + b - a*b
			c = torch.sum(c*s)/torch.sum(c)
			match_scores.append(match_score)
			logits.append(c.view(-1))
		logits = torch.stack(logits, 0)
		return match_scores, logits

	def em(self, h, lengths):
		# get representations with different lengths from the collated single matrix
		e = [None] * len(lengths)
		for i in range(len(lengths)):
			e[i] = h[i, :lengths[i]]
		return e

Overwriting RNABERT/module.py


In [92]:
%%writefile RNABERT/MLM_SFP.py
import random
import time
import numpy as np
import torch
import torch.optim as optim
from torchvision import transforms, datasets
import copy
from Bio import SeqIO
import argparse
from utils.bert import get_config, BertModel, set_learned_params, BertForMaskedLM, visualize_attention, show_base_PCA, fix_params
from module import Train_Module
from dataload import DATA, MyDataset
import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.cluster import adjusted_rand_score
import os
import time
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, completeness_score, homogeneity_score
import torch.nn.functional as F
from sklearn.cluster import MiniBatchKMeans, KMeans, AgglomerativeClustering, SpectralClustering
import itertools

# import alignment_C as Aln_C

random.seed(10)
torch.manual_seed(1234)
np.random.seed(1234)

parser = argparse.ArgumentParser(description='RNABERT')
parser.add_argument('--mag',  type=int, default=1,
                    help='enumerate')
parser.add_argument('--epoch', '-e', type=int, default=200,
                    help='Number of sweeps over the dataset to train')
parser.add_argument('--batch', '-b', type=int, default=20,
                    help='Number of batch size')
parser.add_argument('--maskrate', '-m', type=float, default=0.0,
                    help='mask rate')
parser.add_argument('--pretraining', '-pre', type=str, help='use pretrained weight')
parser.add_argument('--outputweight', type=str, help='output path for weights')
parser.add_argument('--algorithm', type=str, default="global", help='algorithm method')
parser.add_argument('--data_mlm', '-d', type=str, nargs='*', help='data for mlm training')
parser.add_argument('--data_mul', type=str, nargs='*', help='data for mul training')
parser.add_argument('--data_alignment', type=str, nargs='*', help='data for alignment test')
parser.add_argument('--data_clustering', type=str, nargs='*', help='data for clustering test')
parser.add_argument('--data_showbase', type=str, nargs='*', help='data for base embedding')
parser.add_argument('--data_embedding', type=str, nargs='*', help='data for base embedding')
parser.add_argument('--embedding_output', type=str, nargs='*', help='output file for base embedding')
parser.add_argument('--show_aln', action='store_true')

args = parser.parse_args()
batch_size = args.batch
current_time = datetime.datetime.now()

print("start...")
class TRAIN:
    """The class for controlling the training process of SFP"""
    def __init__(self, config):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.module = Train_Module(config)

    def model_device(self, model):
        print("device: ", self.device)
        print('-----start-------')
        model.to(self.device)
        if self.device == 'cuda':
            model = torch.nn.DataParallel(model) # make parallel
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        return model

    def train_MLM_SFP(self, model, optimizer, dl_MLM_SFP, num_epochs, task_type):
        for epoch in range(num_epochs):
            model.train()
            epoch_mlm_loss = 0.0
            epoch_ssl_loss = 0.0
            epoch_mlm_correct = 0.0
            epoch_ssl_correct = 0.0
            epoch_sfp_loss=0.0
            epoch_sfp_correct = 0.0
            epoch_mul_loss = 0.0

            iteration = 1
            t_epoch_start = time.time()
            t_iter_start = time.time()
            data_num = 0
            for batch in dl_MLM_SFP:
                optimizer.zero_grad()
                if task_type == "MLM" or task_type == "SFP":
                    low_seq_0, masked_seq_0, family_0, seq_len_0, low_seq_1, masked_seq_1, family_1, seq_len_1 = batch
                elif task_type == "MUL":
                    low_seq_0, masked_seq_0, family_0, seq_len_0, low_seq_1, masked_seq_1, family_1, seq_len_1, common_index_0, common_index_1 = batch

                masked_seq_0 = masked_seq_0.to(self.device)
                low_seq_0 = low_seq_0.to(self.device)
                masked_seq_1 = masked_seq_1.to(self.device)
                low_seq_1 = low_seq_1.to(self.device)

                masked_seq = torch.cat((masked_seq_0, masked_seq_1), axis=0)
                prediction_scores, prediction_scores_ss, encoded_layers =  model(masked_seq)
                prediction_scores0, prediction_scores1 = torch.split(prediction_scores, int(prediction_scores.shape[0]/2))
                prediction_scores_ss0, prediction_scores_ss1 = torch.split(prediction_scores_ss, int(prediction_scores_ss.shape[0]/2))
                encoded_layers0, encoded_layers1 = torch.split(encoded_layers, int(encoded_layers.shape[0]/2))

                loss = 0
                # MLM LOSS
                mlm_loss_0, mlm_correct_0 = self.module.train_MLM(low_seq_0, masked_seq_0, prediction_scores0)
                mlm_loss_1, mlm_correct_1 = self.module.train_MLM(low_seq_1, masked_seq_1, prediction_scores1)
                mlm_loss = (mlm_loss_0 + mlm_loss_1)/2
                mlm_loss = torch.tensor(0.0) if  torch.isnan(mlm_loss) else mlm_loss
                mlm_correct = (mlm_correct_0 + mlm_correct_1)/2
                epoch_mlm_loss += mlm_loss.item() * batch_size
                epoch_mlm_correct += mlm_correct
                if task_type == "MLM":
                    loss += mlm_loss

                # SFP LOSS
                if task_type == "SFP":
                    z0_list, z1_list =  self.module.em(encoded_layers0, seq_len_0), self.module.em(encoded_layers1, seq_len_1)
                    sfp_loss, sfp_correct = self.module.train_SFP(low_seq_0, seq_len_0, low_seq_1, seq_len_1, family_0, family_1, z0_list, z1_list)
                    sfp_loss = torch.tensor(0.0) if  torch.isnan(sfp_loss) else sfp_loss
                    epoch_sfp_loss += sfp_loss.item()* batch_size
                    epoch_sfp_correct += sfp_correct
                    loss += sfp_loss

                # MULTIPLE LOSS
                if task_type == "MUL":
                    common_index_0 = common_index_0.to(self.device)
                    common_index_1 = common_index_1.to(self.device)
                    z0_list, z1_list =  self.module.em(encoded_layers0, seq_len_0), self.module.em(encoded_layers1, seq_len_1)
                    mul_loss = self.module.train_MUL(z0_list, z1_list, common_index_0, common_index_1, seq_len_0, seq_len_1)
                    mul_loss = torch.tensor(0.0) if  torch.isnan(mul_loss) else mul_loss
                    epoch_mul_loss += mul_loss.item()
                    loss +=  mul_loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                torch.cuda.empty_cache()

            t_epoch_finish = time.time()
            epoch_mlm_loss = epoch_mlm_loss / len(dl_MLM_SFP.dataset)
            epoch_mlm_correct = epoch_mlm_correct / len(dl_MLM_SFP)
            epoch_sfp_loss = epoch_sfp_loss  / len(dl_MLM_SFP.dataset)
            epoch_sfp_correct = epoch_sfp_correct / len(dl_MLM_SFP.dataset)
            epoch_mul_loss = epoch_mul_loss
            print('Epoch {}/{} | MLM Loss: {:.4f} MLM Acc: {:.4f}| SFP Loss: {:.4f} SFP Acc: {:.4f}| MUL Loss: {:.4f}| time: {:.4f} sec.'.format(epoch+1, num_epochs,
                                                                        epoch_mlm_loss, epoch_mlm_correct, epoch_sfp_loss, epoch_sfp_correct, epoch_mul_loss, time.time() - t_epoch_start))
            t_epoch_start = time.time()
        if args.outputweight:
            torch.save(model.state_dict(), args.outputweight + '{0:%m_%d_%H_%M}'.format(current_time))
            torch.save(model.state_dict(), args.outputweight)
        return model

    # make feature vector
    def make_feature(self, model, dataloader, seqs):
        model.eval()
        torch.backends.cudnn.benchmark = True
        batch_size = dataloader.batch_size
        encoding = []
        for batch in dataloader:
            data, label, seq_len= batch
            inputs = data.to(self.device)
            prediction_scores, prediction_scores_ss, encoded_layers =  model(inputs)
            encoding.append(encoded_layers.cpu().detach().numpy())
        encoding = np.concatenate(encoding, 0)

        embedding = []
        for e, seq in zip(encoding, seqs):
            embedding.append(e[:len(seq)].tolist())

        return embedding

    def validateOnCompleteTestData(self, test_loader, simirality_matrix):
        # accuracy and rand index
        nmi = normalized_mutual_info_score
        ari = adjusted_rand_score
        homo = homogeneity_score
        com = completeness_score
        true_labels = np.concatenate([d[1].cpu().numpy() for i,d in enumerate(test_loader)], 0)

        # km = KMeans(n_clusters=len(np.unique(true_labels)), n_init=20, n_jobs=4)
        # y_pred = km.fit_predict(simirality_matrix)

        # ac = AgglomerativeClustering(n_clusters=len(np.unique(true_labels)), affinity='precomputed', linkage='average')
        # ac = AgglomerativeClustering(n_clusters=None,affinity='precomputed', linkage='average', distance_threshold=0.45)
        # y_pred = ac.fit_predict(1+ (-1 * simirality_matrix))

        # y_pred = y_pred.tolist()
        # true_labels = true_labels.tolist()
        # import collections
        # c = collections.Counter(y_pred)
        # y_pred_new = []
        # true_labels_new = []
        # for i, j in zip(y_pred, true_labels):
        #     if c[i] >= 2:
        #         y_pred_new.append(i)
        #         true_labels_new.append(j)
        # print(len(y_pred_new))
        # y_pred = np.array(y_pred_new)
        # true_labels = np.array(true_labels_new)

        sc=SpectralClustering(n_clusters=len(np.unique(true_labels)))
        y_pred=sc.fit(simirality_matrix).labels_

        print(' '*8 + '|==>  nmi: %.4f ,  ari: %.4f,  com: %.4f,  homo: %.4f  <==|'
                      % (nmi(true_labels, y_pred), ari(true_labels, y_pred), com(true_labels, y_pred), homo(true_labels, y_pred)))
        return ari(true_labels, y_pred)

    def align(self, model, dl):
        model.eval()
        pred_match = 0
        ref_match = 0
        TP = 0
        for batch in dl:
            low_seq_0, masked_seq_0, family_0, seq_len_0, low_seq_1, masked_seq_1, family_1, seq_len_1, common_index_0, common_index_1 = batch
            low_seq_0 = low_seq_0.to(self.device)
            low_seq_1 = low_seq_1.to(self.device)
            low_seq = torch.cat((low_seq_0, low_seq_1), axis=0)

            start = time.time()
            prediction_scores, prediction_scores_ss, encoded_layers =  model(low_seq)
            elapsed_time = time.time() - start
            # print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

            prediction_scores0, prediction_scores1 = torch.split(prediction_scores, int(prediction_scores.shape[0]/2))
            encoded_layers0, encoded_layers1 = torch.split(encoded_layers, int(encoded_layers.shape[0]/2))
            z0_list, z1_list =  self.module.em(encoded_layers0, seq_len_0), self.module.em(encoded_layers1, seq_len_1)
            len_TP, len_pred_match, len_ref_match = self.module.test_align(low_seq_0, low_seq_1, z0_list, z1_list, common_index_0, common_index_1, seq_len_0, seq_len_1, args.show_aln)
            TP += len_TP
            pred_match += len_pred_match
            ref_match += len_ref_match

        PPV = TP /pred_match
        sens = TP / ref_match
        f1 = 2 * PPV * sens/(PPV + sens)
        if args.show_aln == False:
            print("alignment accuracy : ", f1, "sens : ", sens, "PPV : ", PPV)
        return f1

    def test(self, ds, test_loader, model):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model.eval()
        data_num = len(test_loader.dataset)
        simirality_matrix = []
        for i in range(data_num):
            single_seq = MyDataset("CLU", np.tile(ds.low_seq[i],(data_num,1)), np.tile(ds.low_seq[i],(data_num,1)),np.tile(ds.family[i],(data_num,1)), np.tile(ds.seq_len[i], data_num))
            single_seq = torch.utils.data.DataLoader(single_seq, batch_size, shuffle=False)
            low = []
            for data0, data1 in zip( test_loader, single_seq):
                x0, label0, seq_len_0 = data0
                x1, label1, seq_len_1 = data1
                x0, label0 = x0.to(device),label0.to(device),
                x1, label1 = x1.to(device),label1.to(device),
                x = torch.cat((x0, x1), axis=0)
                prediction_scores, prediction_scores_ss, encoded_layers =  model(x)
                encoded_layers0, encoded_layers1 = torch.split(encoded_layers, int(encoded_layers.shape[0]/2))
                z0_list, z1_list =  self.module.em(encoded_layers0, seq_len_0), self.module.em(encoded_layers1, seq_len_1)
                _, logits = self.module.match(z0_list, z1_list)
                low.append(torch.squeeze(logits).to('cpu').detach().numpy().copy())
            simirality_matrix.append(np.concatenate(low, 0))
        currentAcc = self.validateOnCompleteTestData(test_loader, np.array(simirality_matrix))
        return currentAcc

def objective():
    config.hidden_size = config.num_attention_heads * config.multiple
    train = TRAIN(config)
    model = BertModel(config)
    model = BertForMaskedLM(config, model)
    if args.data_mlm:
        config.adam_lr = 2e-4
    # if args.data_sfp:
    #     model = fix_params(model)
    #     config.adam_lr = config.adam_lr * 0.5
    if args.data_mul:
        # model = fix_params(model)
        config.adam_lr = 1e-4
    model = train.model_device(model)
    if args.pretraining:
        state_dict = torch.load(args.pretraining, map_location=torch.device(train.device))
        new_state_dict = {key.lstrip('module.'): value for key, value in state_dict.items()}
        model.load_state_dict(new_state_dict,  strict=False)
    optimizer = optim.AdamW([{'params': model.parameters(), 'lr': config.adam_lr}])
    return model , optimizer, train, config

config = get_config(file_path = "./RNA_bert_config.json")
data = DATA(args, config)
model, optimizer, train, config = objective()

#now start training
if args.data_mlm:
    dl_MLM = data.load_data_MLM_SFP(args.data_mlm)
    model = train.train_MLM_SFP(model, optimizer, dl_MLM, args.epoch, "MLM")
# elif args.data_sfp:
#     dl_SFP = data.load_data_MLM_SFP(args.data_sfp)
#     model = train.train_MLM_SFP(model, optimizer, dl_SFP, args.epoch, "SFP")
if args.data_mul:
    dl_MUL = data.load_data_MUL(args.data_mul, "MUL")
    model = train.train_MLM_SFP(model, optimizer, dl_MUL, args.epoch, "MUL")

if args.data_alignment:
    dl_alignment = data.load_data_MUL(args.data_alignment, "MUL")
    alignment_accuracy = train.align(model, dl_alignment)
elif args.data_clustering:
    _, _, ds, test_dl = data.load_data_CLU(args.data_clustering)
    train.test(ds, test_dl, model)

if args.data_showbase:
    seqs, label, SS,  ds, test_dl  = data.load_data_SHOW(args.data_showbase)
    features = train.make_feature(model, test_dl)
    features = features.reshape(-1, features.shape[2])
    show_base_PCA(features, label.reshape(-1), SS)

if args.data_embedding:
    seqs, label, test_dl  = data.load_data_EMB(args.data_embedding)
    features = train.make_feature(model, test_dl, seqs)
    for i, data_set in enumerate(args.embedding_output):
        with open(data_set, 'w') as f:
            for d in features:
                f.write(str(d) + '\n')

Overwriting RNABERT/MLM_SFP.py


In [103]:
#install python 3.9 and dev utils
#you may not need all the dev libraries, but I haven't tested which aren't necessary.
!sudo apt-get update -y
!sudo apt-get install python3.9 python3.9-dev python3.9-distutils libpython3.9-dev

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

#Check that it points at the right location
!python3 --version

# install pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall

#install colab's dependencies
!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

# link to the old google package
!ln -s /usr/local/lib/python3.8/dist-packages/google \
       /usr/local/lib/python3.9/dist-packages/google

# There has got to be a better way to do this...but there's a bad import in some of the colab files
# IPython no longer exposes traitlets like this, it's a separate package now
!sed -i "s/from IPython.utils import traitlets as _traitlets/import traitlets as _traitlets/" /usr/local/lib/python3.9/dist-packages/google/colab/*.py
!sed -i "s/from IPython.utils import traitlets/import traitlets/" /usr/local/lib/python3.9/dist-packages/google/colab/*.py

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,266 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,342 kB]
Fetched 2,950 kB in 2s (1,405 kB/s)
Reading package li

In [105]:
!pip install numpy
!pip3 install torch torchvision
!pip install biopython
!pip install biopython==1.76
!pip install attrdict
!pip install matplotlib
!pip install sklearn
!pip install scikit-learn
!pip install forgi

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/75/cd/7ae0f2cd3fc68aea6cfb2b7e523842e1fa953adb38efabc110d27ba6e423/numpy-1.26.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 43.7 MB/s eta 0:00:00
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/d3/fa/93f3ef65dee8947d8f54eb876ab57a8b019845a45dc07546e2ac214da97b/torch-2.1.0-cp39-cp39-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/8b/64/00e53316beb2f19edcaa0eb283b6087cb4d5275112f24b8ea0d8b49c1d5f/torchvision-0.16.0-cp39-cp39-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/5e/5d/97afbafd9d584ff1b45fcb354a479a3609bd97f912f8f1f6c563cb1fae21/filelock-3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.4 MB/s eta 0:00:00
  Using cached biopython-1.76.tar.gz (16.3 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for biopython: filename=biopython-1.76-cp39-cp39-linux_x86_64.whl size=2603383 sha256=767b3cd713e0ec28052782d48b9805e13f206669b5aeb1ea71190e714e35c5c7
  Stored in directory: /root/.cache/pip/wheels/97/1c/09/f3cca391e94687de244194019727d61885b4c252bb2d8b1588
Successfully built biopython
  Attempting uninstall: biopython
    Found existing installation: biopython 1.81
    Uninstalling biopython-1.81:
      Successfully uninstalled biopython-1.81


  Using cached attrdict-2.0.1-py2.py3-none-any.whl (9.9 kB)
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/e0/8b/b62bc50b01bb2d4af96bc0045c39d60209e2701e172789ceace20a0866b2/matplotlib-3.8.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/2b/c0/24c34c41a180f875419b536125799c61e2330b997d77a5a818a3bc3e08cd/contourpy-1.1.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/06/19/05e6d60206300d030948c2e09c28ad7f6e3c6e2299b9a5beb01261b38f0a/fonttools-4.43.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.wh

  Using cached sklearn-0.0.post9.tar.gz (3.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/af/ad/329a88013936e4372181c0e275c19aa6130b0835876726944b811af5a856/scikit_learn-1.3.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/88/8c/9d1f74196c296046af1f20e6d3fc7fbb27387282315e1643f450bba14329/scipy-1.11.3-cp39-cp39-manylinux_2_17_x86_64.manylinu

In [97]:
!cd RNABERT && wget https://raw.githubusercontent.com/AnzorGozalishvili/NASA_ODSR_DATA/main/bert_mul_2.pth

--2023-10-08 10:28:21--  https://raw.githubusercontent.com/AnzorGozalishvili/NASA_ODSR_DATA/main/bert_mul_2.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2084567 (2.0M) [application/octet-stream]
Saving to: ‘bert_mul_2.pth’

bert_mul_2.pth      100%[===================>]   1.99M  --.-KB/s    in 0.08s   

2023-10-08 10:28:21 (25.2 MB/s) - ‘bert_mul_2.pth’ saved [2084567/2084567]



In [104]:
!cd RNABERT && \
python3 MLM_SFP.py \
    --pretraining bert_mul_2.pth \
    --data_embedding  sample/mlm/RF00001.fa \
    --embedding_output ./predictions.txt \
    --batch 40

Traceback (most recent call last):
  File "/content/RNABERT/MLM_SFP.py", line 3, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'
